In [1]:
import os
from datetime import datetime

import yaml
from IPython.display import display, Markdown
from git import Repo as RepoLoader  # gitpython

from tqdm import tqdm

import papers_analyser.github as github
import papers_analyser.db as db
from papers_analyser.paper import load_papers,download_json_files

%load_ext autoreload
%autoreload 2

In [2]:
project_dir = os.path.abspath("..")

## Connect to DB

In [3]:
connection = db.create_connection( project_dir + "/data/paper.db")
db.create_table_if_not_exist(connection)

2.6.0


## Github API auth-token

In [4]:
auth_token = github.load_auth_token("config.yaml")

C:\Users\Peter-NB\Dev\reproduce-ml-papers\papers_analyser\github.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open(path, "r"))


# Loading papers

In [5]:
json_path = project_dir + "/data"

Download JSON file if you have not done this already.

In [6]:
download_json_files(json_path)

Load json files and create paper objects that map to repositories.

In [6]:
since_date = datetime.strptime("2019-01-01", '%Y-%m-%d')


In [7]:
papers= load_papers(project_dir + "/data",  since_date)


In [8]:
papers_persisted = db.get_papers(connection)

In [9]:
papers_to_scrape = [paper for paper in papers if paper.url not in papers_persisted]

Scrape the metadata, file-names and the whole readme-file from the github-api for each repo.
If you have a lot of repos to scrape, you have to rerunt his cell multiple times.

In [ ]:
for paper in tqdm(reversed(papers_to_scrape)):
    for repo in paper.repos:
        if not repo.scraped:
            repo.scrape(auth_token)

221it [31:39, 10.65s/it]

404 Client Error: Not Found for url: https://api.github.com/repos/rsummers11/CADLab/tree/master/Lung_Segmentation_XLSor
404 Client Error: Not Found for url: https://api.github.com/repos/google/brain-tokyo-workshop/tree/master/WANNRelease
404 Client Error: Not Found for url: https://api.github.com/repos/google-research/google-research/tree/master/weak_disentangle
404 Client Error: Not Found for url: https://api.github.com/repos/pytorch/fairseq/blob/master/fairseq/models/wav2vec.py
404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/models/tree/master/research/struct2depth
404 Client Error: Not Found for url: https://api.github.com/repos/deepmind/deepmind-research/tree/master/transporter
404 Client Error: Not Found for url: https://api.github.com/repos/tensorflow/models/tree/master/research/delf
404 Client Error: Not Found for url: https://api.github.com/repos/google-research/language/tree/master/language/bert_extraction
404 Client Error: Not Found for url: https

## Take a look on README files

In [22]:
readmes = [paper.repo[0].readme for paper in papers]

NameError: name 'papers' is not defined

In [36]:
display(Markdown(readmes[4]))

NameError: name 'readmes' is not defined

# Save to database

In [37]:
i=1
j=1
unscraped = list()
for paper in papers_to_scrape:
    if paper.repos_scraped():
        db.insert_paper(connection,paper)
    else:
        unscraped.append(paper)

connection.commit()

In [23]:
unscraped[0].repos_scraped()

True

In [ ]:
len(papers_to_scrape)

In [28]:
connection.close()

## Clone repo

In [36]:
dir = driverPath = project_dir + "/repos"
for paper in papers:
    paper_path = dir + "/" + paper.title.replace( ":","")
    if not os.path.exists(paper_path):
        os.mkdir(paper_path)
        
    for repo in paper.repo:
        repo_path = paper_path+ repo.repo_name.replace("/","_")
        
        if not os.path.exists(repo_path):
            os.mkdir(repo_path)
            
        RepoLoader.clone_from(repo.clone_url,repo_path)
    

